In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Dropout, MaxPooling2D,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

training_size = 300000
batch_size = 128
epochs = 15
IMG_HEIGHT = 60
IMG_WIDTH = 28

CLASS_NAMES = np.array([])
for i in range(10):
    CLASS_NAMES =np.append(CLASS_NAMES,chr(ord("0")+i))
for i in range(26):
    CLASS_NAMES = np.append(CLASS_NAMES,chr(ord("A")+i))

#training dir / validation dir
train_dir_data01 = os.path.abspath(os.getcwd())+"\\train\\data01_train\\"
val_dir_data01 = os.path.abspath(os.getcwd())+"\\dev\\data01_dev"
num_train_data01 = len(os.listdir(train_dir_data01))
num_val_data01 = len(os.listdir(val_dir_data01))
print("Train data count: " + str(num_train_data01))
print("Validation data count: " + str(num_val_data01))

#store filename and label to a Dataframe 
import pandas as pd
data = pd.read_csv("train\\data01_train.csv")
for i in range(len(data)):
    data.iloc[i,1] = list(data.iloc[i,1])

#maybe there's an easier way to separate labels??
arr = np.zeros([6,50000],str)
for j in range(6):
    for i in range(len(data)):
        arr[j][i] = data.iloc[i,1][j]
data = data.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

data

Train data count: 50000
Validation data count: 10000


,filename,code0,code1,code2,code3,code4,code5
0,000000.jpg,Z,3,2,7,0,D
1,000001.jpg,V,M,Y,X,E,8
2,000002.jpg,J,Z,K,W,V,U
3,000003.jpg,X,9,I,2,7,H
4,000004.jpg,H,5,X,G,R,2
...,...,...,...,...,...,...,...
49995,049995.jpg,3,Y,F,I,S,E
49996,049996.jpg,4,S,U,T,F,7
49997,049997.jpg,C,Y,U,N,0,9
49998,049998.jpg,K,C,D,8,I,O


In [2]:
#no crop function
def parse_label(strIN):
    return (strIN==CLASS_NAMES).astype(float)
def readimg_to_tensor(fn):
    a = tf.io.read_file((fn))
        #crop to [60,30,3]
    img = tf.io.decode_jpeg(a)
    return img

In [3]:
#to one hot
for i in range(6):
    data.iloc[:,i+1] = data.iloc[:,i+1].map(parse_label)
data

,filename,code0,code1,code2,code3,code4,code5
0,000000.jpg,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,000001.jpg,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
2,000002.jpg,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,000003.jpg,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,000004.jpg,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...
49995,049995.jpg,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
49996,049996.jpg,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
49997,049997.jpg,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
49998,049998.jpg,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [4]:
TL = tf.convert_to_tensor(list(data.iloc[:,1].values))
TL2 = tf.convert_to_tensor(list(data.iloc[:,2].values))
TL3 = tf.convert_to_tensor(list(data.iloc[:,3].values))
TL4 = tf.convert_to_tensor(list(data.iloc[:,4].values))
TL5 = tf.convert_to_tensor(list(data.iloc[:,5].values))
TL6 = tf.convert_to_tensor(list(data.iloc[:,6].values))

In [5]:
#no crop
train_dir = tf.data.Dataset.list_files(train_dir_data01+'*.jpg',shuffle=False) 
train_data = train_dir.map(lambda x: readimg_to_tensor(x))
train_label = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))
train_data01=tf.data.Dataset.zip((train_data,train_label))

In [6]:
#validation data no crop
val_dir = tf.data.Dataset.list_files(val_dir_data01+'*.jpg',shuffle=False) 
val_data01 = val_dir.map(lambda x: readimg_to_tensor(x))

In [7]:
#railway model
print("Creating CNN model...")
inn = Input((60, 200, 3))
out = inn
out = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(out)
out = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Dropout(0.3)(out)
out = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(out)
out = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Dropout(0.3)(out)
out = Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu')(out)
out = Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Dropout(0.3)(out)
out = Conv2D(filters=256, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Flatten()(out)
sep = Dropout(0.3)(out)
dig1 = Dense(36, name='digit1', activation='softmax')(sep)
dig2 = Dense(36, name='digit2', activation='softmax')(sep)
dig3 = Dense(36, name='digit3', activation='softmax')(sep)
dig4 = Dense(36, name='digit4', activation='softmax')(sep)
dig5 = Dense(36, name='digit5', activation='softmax')(sep)
dig6 = Dense(36, name='digit6', activation='softmax')(sep)
model = tf.keras.models.Model(inputs=inn, outputs=[dig1, dig2,dig3,dig4,dig5,dig6])
model.compile(
    loss=[
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
    ],
    optimizer='adam', 
    metrics=['accuracy'])
model.summary()

Creating CNN model...
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 60, 200, 3)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 60, 200, 32)  896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 58, 198, 32)  9248        conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 58, 198, 32)  128         conv2d_1[0][0]                   
________________________________________________________________________

In [8]:
#number of element
num_elements = tf.data.experimental.cardinality(train_data01).numpy()
num_elements

50000

In [12]:
epochs=15
batch_size=128
big_batch_size=5000
train_data_gen=iter(train_data01.repeat().batch(big_batch_size))

for i in range(int(num_elements/big_batch_size)):
   img , lb = next(train_data_gen)
   history = model.fit(
       img,
       lb,
       batch_size=batch_size,
       validation_split=0.1,
       shuffle=True,
       use_multiprocessing=True,
       epochs=epochs,
       verbose=2
       )


5986 - val_digit5_loss: 3.5561 - val_digit6_loss: 3.5902 - val_digit1_accuracy: 0.0420 - val_digit2_accuracy: 0.0360 - val_digit3_accuracy: 0.0440 - val_digit4_accuracy: 0.0380 - val_digit5_accuracy: 0.0400 - val_digit6_accuracy: 0.0220
Epoch 8/15
36/36 - 7s - loss: 19.8060 - digit1_loss: 3.3175 - digit2_loss: 3.2603 - digit3_loss: 3.2790 - digit4_loss: 3.3124 - digit5_loss: 3.3142 - digit6_loss: 3.3227 - digit1_accuracy: 0.0827 - digit2_accuracy: 0.1049 - digit3_accuracy: 0.1004 - digit4_accuracy: 0.0916 - digit5_accuracy: 0.0889 - digit6_accuracy: 0.0891 - val_loss: 21.4805 - val_digit1_loss: 3.5796 - val_digit2_loss: 3.6028 - val_digit3_loss: 3.6153 - val_digit4_loss: 3.5859 - val_digit5_loss: 3.5392 - val_digit6_loss: 3.5577 - val_digit1_accuracy: 0.0520 - val_digit2_accuracy: 0.0320 - val_digit3_accuracy: 0.0360 - val_digit4_accuracy: 0.0360 - val_digit5_accuracy: 0.0340 - val_digit6_accuracy: 0.0280
Epoch 9/15
36/36 - 7s - loss: 19.5265 - digit1_loss: 3.2763 - digit2_loss: 3.1973

KeyboardInterrupt: 

In [65]:
def parseResult(RS):
    for j in range(len(RS)):
        PredChr=''
        for i in range(6):
            PredChr+= CLASS_NAMES[(np.where(RS[j][i] == RS[j][i].max())[0][0])]
        if j == 0:
            LB = [PredChr]
        else:
            LB.append(PredChr)      
    return LB
        

In [88]:
#output to CSV
VDlabel = np.array([])
img = next(iter(val_data01.batch(10000)))
result = model.predict(img,use_multiprocessing = True)
result = np.array(result).reshape((10000,6,36))

In [89]:
val_dir = tf.data.Dataset.list_files(val_dir_data01+'*.jpg',shuffle=False) 
test = parseResult(result)
val_file_name = pd.read_csv("./dev/data01_dev.csv")
val_file_name.join(pd.DataFrame(test,columns=["code"])).to_csv("data01_dev.csv",index = False)